![Hello](https://media1.tenor.com/images/469f17daad56085938aafe44c44d31fd/tenor.gif)

Hello.  I'm Marcus, and I have a data problem. 🍷 DirtyData is blog about bad data and good Data Science.  I don't know eveything but hopefully we will learnt together. 📓 Over the next few posts we will play with categorical data- the sparser the better- we will look at how to build data pipelines and cool techniques in exploring them. 🤓 I am the worst spelling and grammar-checker, so I will appologize for the fact that what I write is unreadable.  As someone living in South Africa and interested in African data problems, we will try chat and get input from local Data Scientists and researchers on what they are learning and what is keeping them exciting.  🌔

I thought it may be fun, as an introduction and for you to get to know me, to explore some of my personal data and thanks to GDPR and Facebook's repent I know own my data and can explore it with you with some fun graphs and visuals.  Hurray! 💥 Check out how to [download](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=4&cad=rja&uact=8&ved=0ahUKEwiMmYj4zYfcAhXUa8AKHRnSBXMQFgg6MAM&url=https%3A%2F%2Fwww.wired.com%2Fstory%2Fdownload-facebook-data-how-to-read%2F&usg=AOvVaw30_I7goGnKq1ltdH4vxU8D) your own.  

I am going to appologie in advanced for importing all these libraries.  📚

In [1]:
import os
import pandas as pd
import numpy as np
from gensim.models.deprecated.old_saveload import SaveLoad

import plotly.plotly as py
import plotly.graph_objs as go

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.base import TransformerMixin
from sklearn.metrics import pairwise_distances

So here are my all the posts I have been tagged in.  There are only around 800, as I am not that popular. 😭 

In [2]:
path = os.path.join(".","Datasets","facebook_posts","other_people's_posts_to_your_timeline.json")

In [3]:
json_file = pd.read_json(path, orient='list')
posts = pd.io.json.json_normalize(json_file.wall_posts_sent_to_you)

posts.data = posts.data.astype(str).apply(lambda x: x[11:-3])
posts.timestamp = pd.to_datetime(posts.timestamp, unit='s')

In [4]:
posts.head()

,attachments,data,tags,timestamp,title
0,NaN,May your year ahead be fuelled with energy and...,NaN,2018-05-15 08:52:55,Fleur Lerwill added a new photo to Marcus Gawr...
1,NaN,Happy birthday my wonderful friend. Have a ma...,NaN,2018-05-13 11:53:53,Geoff Olivier wrote on your timeline.
2,NaN,Happy birthday Marcus!! Hope you had a great day,NaN,2018-05-13 10:18:45,Javan Moodley wrote on your timeline.
3,NaN,"happy birthday bud, hope it's been great!",NaN,2018-05-12 19:35:34,James Combrink wrote on your timeline.
4,NaN,Happy birthday Marcus! Hope you've had a great...,NaN,2018-05-12 18:14:07,Jason W Gaskell wrote on your timeline.


I will be using a cool technique called [Word2Vec](https://www.tensorflow.org/tutorials/word2vec) for word embeddings 🔠.  I will be using a [pretrained embeddings](https://github.com/jhlau/doc2vec#pre-trained-doc2vec-models), as I have neither the time, nor money to learn my own 🏋.  In order to create the document embeddings I will be using a weighting of the [term frequency–inverse document frequencies](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).  

In [5]:
w2v_wiki = SaveLoad.load('./Models/W2V_enwiki/word2vec.bin', mmap='r')

In [6]:
class WeightedWordVectors(TransformerMixin):
    """Trains Collation Model to join co-occuring works into single entities or concepts
    
    Libraries & Versions:
    Python==3.6.5
    Pandas=='0.23.1' as pd
    nltk=='3.3'
    numpy=='1.14.5'
    
    Keyword arguments:
    X -- Pandas Series of text as strings
    """
    def __init__(self, model=None):
        self.word2vec =  model
        self.tfidf = TfidfVectorizer(vocabulary= list(self.word2vec.vocab.keys()))
        pass
    
    def fit(self, X, y=None):
        self.tfidf.fit(X)
        return self

    def transform(self, X):
        self.weighted_docs = []
        
        for doc in X:
            self.t = self.tfidf.transform([doc])
            self.words = np.array(list(self.tfidf.vocabulary_.keys()))[self.t.nonzero()[1]]
            self.weights = self.t[0,self.t.nonzero()[1]].todense()
            
            self.weighted_doc = np.empty((300,), float)
            for word, weight in zip(self.words.tolist(), self.weights.tolist()[0]):
                try:
                    self.weighted_doc = self.weighted_doc + w2v_wiki[word] * weight
                except:
                    pass
                
            self.weighted_docs.append(self.weighted_doc.tolist())
                
        return(pd.DataFrame(self.weighted_docs).fillna(0))

In [7]:
weighter = WeightedWordVectors(w2v_wiki)

In [8]:
document_vectors = weighter.fit_transform(posts.data)

In [9]:
document_similarities = pairwise_distances(document_vectors.astype(np.float64), metric='cosine', n_jobs=-2)

A common tool in visualizing the document vectors is using T-distributed Stochastic Neighbour Embeddings to convert the document similarities into two components which maintain the overall structure of the documents.  🌳

In [10]:
document_tsne = TSNE(metric="precomputed")
document_comp = document_tsne.fit_transform(document_similarities)

document_comp = pd.DataFrame(document_comp, columns=['x','y'])

LET'S CHECK THE PLOT! 📊 I've tried to make it a bit interactive so you can see the grouped posts.  ENJOY.  

In [11]:
py.iplot([go.Scatter(x=document_comp.x.tolist(), y=document_comp.y.tolist(),  mode = 'markers', opacity=0.5, text=posts.data.tolist())])